# NeVA Training / Inference Tutorial

### Note:
Currently, this notebook must be run in a NeMo container. An example command to launch the container:

```
docker run --gpus all -it --rm -v <your_nemo_dir>:/opt/NeMo --shm-size=8g \
     -p 8888:8888 --ulimit memlock=-1 --ulimit \
      stack=67108864 <your_nemo_container>
```

## Introduction

This notebook illustrates how to train and perform inference using NeVA with the NeMo Toolkit. NeVA originates from [LLaVA](https://github.com/haotian-liu/LLaVA) (Large Language and Vision Assistant) and is a powerful multimodal image-text instruction tuned model optimized within the NeMo Framework. 

This tutorial will guide you through the following topics:
1. Prepare pre-requisites for NeVA training
2. Training a NeVA model
3. Performing inference with the trained model

## Datasets

Please refer to [NeMo User Guide](https://docs.nvidia.com/nemo-framework/user-guide/latest/multimodalmodels/multimodallanguagemodel/neva/dataprep.html#prepare-pretraining-and-fine-tuning-datasets) for preparing NeVA dataset for pretrain and fine-tuning.


### Pre-Training Dataset

The pre-training dataset is open-sourced from the LLaVA implementation and can be downloaded [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Pretrain). The dataset consists of a 558K subset of the LAION-CC-SBU dataset with BLIP captions.

The associated images for pretraining can be downloaded via HuggingFace [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Pretrain/blob/main/images.zip).

### Instruction Tuning Dataset

The instruction tuning annotations are sourced from the LLaVA implementation and are available [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/blob/main/llava_v1_5_mix665k.json).

The associated images for the mixture instruction tuning annotations can be found [here](https://github.com/haotian-liu/LLaVA?tab=readme-ov-file#visual-instruction-tuning). After extracting, the data should be formatted as follows:

```
    images
      ├── coco
      │    └── train2017
      ├── gqa
      │    └── images
      ├── ocr_vqa
      │    └── images
      ├── textvqa
      │    └── train_images
      └── vg
           ├── VG_100K
           └── VG_100K_2
```

After downloading all below datasets for pretraining and instruction tuning, please put data folder at `/workspace/datasets`. Your dataset directory should look something similar to:

```
LLaVA-Pretrain-LCS-558K
├── blip_laion_cc_sbu_558k.json
├── images
LLaVA-Instruct-mixture
├── llava_v1_5_mix665k.json
└── images
    └── ...
```

## Setting up Checkpoint and Tokenizer

In this notebook, we first need to convert the Vicuna 1.5 checkpoint into the .nemo format. Meanwhile, special tokens must be incorporated into the tokenizer for NeVA training. After downloading language models from Hugging Face, ensure you also fetch the corresponding tokenizer model. Using the 7B-chat model as a reference.

In [ ]:
! mkdir -p /workspace/checkpoints

# Download vicuna checkpoint from HF
! git clone https://huggingface.co/lmsys/vicuna-7b-v1.5 /workspace/checkpoints/vicuna-7b-v1.5

# Convert checkpoint
! python /opt/NeMo/scripts/checkpoint_converters/convert_llama_hf_to_nemo.py \
  --input_name_or_path /workspace/checkpoints/vicuna-7b-v1.5 \
  --output_path /workspace/checkpoints/vicuna-7b-v1.5.nemo

# Prepare tokenizer
! cd /opt && git clone https://github.com/google/sentencepiece.git && \
  cd sentencepiece && \
  mkdir build && \
  cd build && \
  cmake .. && \
  make && \
  make install && \
  ldconfig && \
cd /opt/sentencepiece/src/ && protoc --python_out=/opt/NeMo/scripts/tokenizers/ sentencepiece_model.proto

! python /opt/NeMo/scripts/tokenizers/add_special_tokens_to_sentencepiece.py \
--input_file /workspace/checkpoints/vicuna-7b-v1.5/tokenizer.model \
--output_file /workspace/checkpoints/vicuna-7b-v1.5/tokenizer_neva.model \
--is_userdefined \
--tokens "<extra_id_0>" "<extra_id_1>" "<extra_id_2>" "<extra_id_3>" \
         "<extra_id_4>" "<extra_id_5>" "<extra_id_6>" "<extra_id_7>"


## Training

### Feature Alignment Pre-Training

We provide a set of scripts for pre-training and fine-tuning which can be kicked off with CLI flags defining specified arguments. 

An example of a pre-training script execution (note the scripts will only perform 100 steps with a small micro batch size, this is not a full training):

In [ ]:
! torchrun --nproc_per_node=4 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_pretrain.py \
 ++cluster_type=BCP \
 trainer.precision=bf16 \
 trainer.num_nodes=1 \
 trainer.devices=4 \
 trainer.val_check_interval=50 \
 trainer.limit_val_batches=5 \
 trainer.log_every_n_steps=1 \
 trainer.max_steps=100 \
 model.megatron_amp_O2=True \
 model.micro_batch_size=1 \
 model.global_batch_size=4 \
 model.tensor_model_parallel_size=1 \
 model.pipeline_model_parallel_size=1 \
 model.mcore_gpt=True \
 model.transformer_engine=True \
 model.data.data_path=/workspace/datasets/LLaVA-Pretrain-LCS-558K/blip_laion_cc_sbu_558k.json \
 model.data.image_folder=/workspace/datasets/LLaVA-Pretrain-LCS-558K/images \
 model.tokenizer.library=sentencepiece \
 model.tokenizer.model=/workspace/checkpoints/vicuna-7b-v1.5/tokenizer_neva.model \
 model.encoder_seq_length=4096 \
 model.num_layers=32 \
 model.hidden_size=4096 \
 model.ffn_hidden_size=11008 \
 model.num_attention_heads=32 \
 model.normalization=rmsnorm \
 model.do_layer_norm_weight_decay=False \
 model.apply_query_key_layer_scaling=True \
 model.bias=False \
 model.activation=fast-swiglu \
 model.headscale=False \
 model.position_embedding_type=rope \
 model.rotary_percentage=1.0 \
 model.num_query_groups=null \
 model.data.num_workers=0 \
 model.mm_cfg.llm.from_pretrained=/workspace/checkpoints/vicuna-7b-v1.5.nemo \
 model.mm_cfg.llm.model_type=v1 \
 model.data.conv_template=v1 \
 model.mm_cfg.vision_encoder.from_pretrained='openai/clip-vit-large-patch14' \
 model.mm_cfg.vision_encoder.from_hf=True \
 model.optim.name="fused_adam" \
 exp_manager.create_checkpoint_callback=True \
 exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True \
 exp_manager.create_wandb_logger=False

**Note**: To initialize training a model from scratch rather than from a pretrained checkpoint, you may specify `null` instead of a path in the CLI arguments.

### Image-Language Pair Instruction Fine-Tuning

Fine-tuning can also be run from within the container via a similar command leveraging the `neva_finetune.py` script. We leverage the checkpoint saved from pretrain step to further finetune it, given by `model.restore_from_path=/workspace/nemo_experiments/nemo_neva/checkpoints/nemo_neva.nemo`.

An example of an image-text pair instruction tuning script execution (note the scripts will only perform 1000 steps with a small micro batch size, this is not a full training):

In [ ]:
! torchrun --nproc_per_node=4 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_finetune.py \
++cluster_type=BCP \
 trainer.precision=bf16 \
 trainer.num_nodes=1 \
 trainer.devices=4 \
 trainer.val_check_interval=50 \
 trainer.limit_val_batches=50 \
 trainer.max_steps=100 \
 model.restore_from_path=/workspace/nemo_experiments/nemo_neva/checkpoints/nemo_neva.nemo \
 model.megatron_amp_O2=True \
 model.micro_batch_size=1 \
 model.global_batch_size=2 \
 model.tensor_model_parallel_size=4 \
 model.pipeline_model_parallel_size=1 \
 model.mcore_gpt=True \
 model.transformer_engine=True \
 model.data.data_path=/workspace/datasets/LLaVA-Instruct-mixture/llava_v1_5_mix665k.json \
 model.data.image_folder=/workspace/datasets/LLaVA-Instruct-mixture/images \
 model.tokenizer.library=sentencepiece \
 model.tokenizer.model=/workspace/checkpoints/vicuna-7b-v1.5/tokenizer_neva.model \
 model.encoder_seq_length=4096 \
 model.num_layers=32 \
 model.hidden_size=4096 \
 model.ffn_hidden_size=11008 \
 model.num_attention_heads=32 \
 model.normalization=rmsnorm \
 model.do_layer_norm_weight_decay=False \
 model.apply_query_key_layer_scaling=True \
 model.bias=False \
 model.activation=fast-swiglu \
 model.headscale=False \
 model.position_embedding_type=rope \
 model.rotary_percentage=1.0 \
 model.num_query_groups=null \
 model.data.num_workers=0 \
 model.mm_cfg.llm.from_pretrained=/workspace/checkpoints/vicuna-7b-v1.5.nemo \
 model.mm_cfg.llm.model_type=v1 \
 model.data.conv_template=v1 \
 model.mm_cfg.vision_encoder.from_pretrained='openai/clip-vit-large-patch14' \
 model.mm_cfg.vision_encoder.from_hf=True \
 exp_manager.create_checkpoint_callback=True \
 exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True \
 exp_manager.name="nemo_neva_finetune" \
 model.optim.name="fused_adam"

## Inference

### From Pre-trained Checkpoints

If you would like to use NeVA for inference from pre-trained checkpoint via HuggingFace, you can use the checkpoint from fine-tune step or convert from HuggingFace to `.nemo` first. Since we didn't finish full training in this tutorial with NeMo. We will instruct how you can convert a checkpoint from Hugging Face.

In [ ]:
! python3 /opt/NeMo/scripts/checkpoint_converters/convert_llava_hf_to_nemo.py \
    --input_name_or_path llava-hf/llava-1.5-7b-hf \
    --output_path /workspace/checkpoints/llava-7b.nemo \
    --tokenizer_path /workspace/checkpoints/vicuna-7b-v1.5/tokenizer_neva.model

### Running Inference

NeVA inference via the NeMo Framework can be quickly spun up via the NeMo Launcher and a few modifications to use the default NeVA inference config file.

Inference can be run with a similar command leveraging the provided inference script `neva_evaluation.py` within the container.

An example of an inference script execution:

In [ ]:
! echo '{"image": "RTX4080.png", "prompt": "<image>\nCan you describe this image?"}' > sample.jsonl
! mkdir images && wget https://assets.nvidia.partners/images/png/TUF_Gaming_GeForce_RTX_4080_SUPER_OC_edition_packaging_with_card__12419.png --output-document=images/RTX4080.png
! torchrun --nproc_per_node=1 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_evaluation.py \
tensor_model_parallel_size=1 \
pipeline_model_parallel_size=1 \
neva_model_file=/workspace/checkpoints/llava-7b.nemo \
trainer.devices=1 \
trainer.precision=bf16 \
prompt_file=sample.jsonl \
inference.media_base_path=images \
output_file=output.jsonl \
inference.temperature=0.2 \
inference.tokens_to_generate=256